In [ ]:
import pandas as pd
import folium
import numpy as np
from urllib.parse import quote

In [ ]:
df = pd.read_csv('rawdata_final.csv', encoding = 'utf-8')
df.drop(columns = 'Unnamed: 0', inplace = True)

addr_list = []
for k in df['Location']:
    addr_list.append(k)

addr_list

In [ ]:
#주소로 좌표 얻기
import json
import requests
MY_APP_KEY = '3123c75b5738c8b04d18ea0521ee9f02'
addr_xyList = []

for addr in addr_list:
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query='+ quote(addr)
    headers = {"Authorization": 'KakaoAK ' + MY_APP_KEY}
    tempDic = {"addr": addr, "x" : None, "y": None}
    result = json.loads(str(requests.get(url, headers=headers).text))
    address = result['documents']
    if len(address) != 0:
        x = float(address[0]['x'])
        y = float(address[0]['y'])
        tempDic['x'] = x
        tempDic['y'] = y
        addr_xyList.append(tempDic)
    else:
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+ quote(addr)
        headers = {"Authorization": 'KakaoAK ' + MY_APP_KEY}
        tempDic = {"addr": addr, "x" : None, "y": None}
        result = json.loads(str(requests.get(url, headers=headers).text))
        address = result['documents']
        if len(address) == 0:
            continue
        else:
            x = float(address[0]['x'])
            y = float(address[0]['y'])
            tempDic['x'] = x
            tempDic['y'] = y
            addr_xyList.append(tempDic)
addr_xyList

In [ ]:
addr_df = pd.DataFrame(addr_xyList)
freq_df = addr_df.groupby('addr').count().sort_values(by = 'x', ascending = False)
freq_df.drop('x', axis = 1, inplace = True)
freq_df.columns = ['freq']
freq_df.reset_index(inplace = True)

total_df = pd.merge(addr_df, freq_df)
total_df

## 전국 분포

In [ ]:
#circlemarker
cheesecake_map = folium.Map(location=[37.514322572335935, 127.06283102249932],zoom_start=8)
for i in range(len(total_df)):
    x = total_df['x'][i]
    y = total_df['y'][i]
    folium.CircleMarker([y, x], radius = float(total_df['freq'][i]), color = '#FCAE1E', fill_color = '#FFDB58', fill_opacity=0.03, weight=4, opacity=1, tooltip = total_df['addr'][i]).add_to(cheesecake_map)
    
cheesecake_map.save('cheesecake_circle.html')

In [ ]:
#circlemarker서울 확대
seoul_circle_map = folium.Map(location=[37.514322572335935, 127.06283102249932],zoom_start=12)
for i in range(len(total_df)):
    x = total_df['x'][i]
    y = total_df['y'][i]
    folium.CircleMarker([y, x], radius = float(total_df['freq'][i])*3, color = '#FCAE1E', fill_color = '#FFDB58', fill_opacity=0.03, weight=4, opacity=1, tooltip = total_df['addr'][i]).add_to(seoul_circle_map)
    
seoul_circle_map.save('seoul_circle.html')

## 서울시 구별 분포

In [ ]:
geo_path = 'skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [ ]:
guList = []

for k in addr_xyList:
    
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=' + str(k['x']) + '&y=' + str(k['y'])
    headers = {"Authorization": 'KakaoAK ' + MY_APP_KEY}

    result = json.loads(str(requests.get(url, headers=headers).text))
    address = result['documents'][0]['region_2depth_name']
    guList.append(address)

#구별 빈도수로 데이터프레임 생성
gu_df = pd.DataFrame(columns = ['Gu', 'Frequency'])
gu_df['Gu'] = [x['id'] for x in geo_str['features']]

for k in range(len(gu_df)):
    count = guList.count(gu_df['Gu'][k])
    gu_df['Frequency'][k] = count   

gu_df['Frequency'] = gu_df['Frequency'].astype(int)
gu_df['Gu'] = gu_df['Gu'].astype(str)
gu_df.set_index('Gu', inplace = True)

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Terrain')

map.choropleth(geo_data = geo_str,
               data = gu_df.Frequency,
               columns = [gu_df.index, gu_df.Frequency],
               fill_color = 'YlGnBu',
               key_on = 'feature.id') #PuRd, YlGnBu

map.save('cheesecake_area.html')